# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [31]:
#!pip3 install plotly
#!pip3 install chart-studio
#!pip3 install cufflinks
#!pip3 install xlrd

In [48]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import re

cf.go_offline()

In [49]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [50]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [51]:
dataNoUK = data[data['Country'] != "United Kingdom"]
dataNoUK.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
103,538095,2010-12-09 14:55:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,1,2.95,2.95,17097,Spain
147,538826,2010-12-14 12:58:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.20,12370,Cyprus
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE
183,539353,2010-12-17 11:30:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,12782,Portugal
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE


In [52]:
dataNoUK['Year'] = dataNoUK['InvoiceDate'].dt.year
dataNoUK['Month'] = dataNoUK['InvoiceDate'].dt.month
dataNoUK.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month
103,538095,2010-12-09 14:55:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,1,2.95,2.95,17097,Spain,2010,12
147,538826,2010-12-14 12:58:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.20,12370,Cyprus,2010,12
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE,2010,12
183,539353,2010-12-17 11:30:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,12782,Portugal,2010,12
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE,2010,12


In [56]:
dataNoUK1 = dataNoUK[(dataNoUK.Year == 2011)&(dataNoUK.Month == 4)]
dataNoUK1.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month
724,549667,2011-04-11 12:20:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE,2011,4
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France,2011,4
3320,550899,2011-04-21 12:07:00,22752,SET 7 BABUSHKA NESTING BOXES,2,8.50,17.00,13505,Switzerland,2011,4
4880,550527,2011-04-19 10:48:00,84879,ASSORTED COLOUR BIRD ORNAMENT,40,1.69,67.60,12476,Germany,2011,4
4882,550620,2011-04-19 13:39:00,84879,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,12585,Germany,2011,4


In [93]:
dataNoUK1Group = dataNoUK1.groupby('Country').sum()
dataNoUK1Group = dataNoUK1Group.reset_index()
dataNoUK1Group.head()

,Country,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID,Year,Month
0,Australia,9338321,224,40.65,421.60,210681,34187,68
1,Austria,13225072,308,95.18,584.78,298236,48264,96
2,Belgium,60518251,1170,273.38,1788.48,1365606,221210,440
3,Brazil,17606432,356,142.60,1143.60,408608,64352,128
4,Channel Islands,4399312,96,28.10,243.00,119448,16088,32


In [94]:
import plotly.express as px
import plotly.graph_objects as go

In [97]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=dataNoUK1Group.Country,
    y=dataNoUK1Group.Quantity,
    name='Quantity',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=dataNoUK1Group.Country,
    y=dataNoUK1Group.Revenue,
    name='Revenue',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [70]:
dataFra = data[data['Country'] == "France"]
dataFra.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
1379,563502,2011-08-17 08:50:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.60,12722,France
1931,578541,2011-11-24 15:01:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,8,2.95,23.60,14277,France
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France
2706,566816,2011-09-15 10:53:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,12,4.25,51.00,12731,France


In [74]:
dataFra['Year'] = dataFra['InvoiceDate'].dt.year
dataFra['Month'] = dataFra['InvoiceDate'].dt.month
dataFra['Day'] = dataFra['InvoiceDate'].dt.day
dataFra.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France,2011,4,26
1379,563502,2011-08-17 08:50:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.60,12722,France,2011,8,17
1931,578541,2011-11-24 15:01:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,8,2.95,23.60,14277,France,2011,11,24
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France,2011,1,12
2706,566816,2011-09-15 10:53:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,12,4.25,51.00,12731,France,2011,9,15


In [77]:
dataFra1 = dataFra[(dataFra.Year == 2011)&((dataFra.Month >= 1)&(dataFra.Month <= 5))]
dataFra1.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France,2011,4,26
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France,2011,1,12
3290,545181,2011-02-28 14:45:00,22752,SET 7 BABUSHKA NESTING BOXES,4,8.50,34.00,12509,France,2011,2,28
4614,542629,2011-01-31 09:57:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France,2011,1,31
4795,548409,2011-03-31 10:27:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France,2011,3,31


In [123]:
dataFraSum = dataFra1.groupby('Month', as_index=False).sum()

In [124]:
fig1 = go.Figure()
fig1.add_trace(go.Line(
    x=dataFraSum.Month,
    y=dataFraSum.Quantity,
    name='Quantity',
    marker_color='indianred'
))
fig1.add_trace(go.Line(
    x=dataFraSum.Month,
    y=dataFraSum.Revenue,
    name='Revenue',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig1.update_layout(barmode='group', xaxis_tickangle=-45)
fig1.show()

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [106]:
dataParty = data[data['Description'] == "PARTY BUNTING"]
dataParty.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
252864,536956,2010-12-03 12:43:00,47566,PARTY BUNTING,5,4.65,23.25,14210,United Kingdom
252865,537065,2010-12-05 11:57:00,47566,PARTY BUNTING,5,4.65,23.25,12567,France
252866,537128,2010-12-05 12:15:00,47566,PARTY BUNTING,2,4.65,9.30,12841,United Kingdom
252867,537142,2010-12-05 12:57:00,47566,PARTY BUNTING,1,4.65,4.65,12748,United Kingdom
252868,537420,2010-12-06 15:18:00,47566,PARTY BUNTING,5,4.65,23.25,17519,United Kingdom


In [114]:
dataPartyGroup = dataParty.groupby('Country').mean()
dataPartyGroup = dataPartyGroup.reset_index()
dataPartyGroup.head()

,Country,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
0,Australia,554329.625000,33.125000,4.7125,143.78125,12399.250000
1,Austria,552202.000000,8.000000,4.9500,39.60000,12414.000000
2,Belgium,557600.000000,4.000000,4.9500,19.80000,12363.000000
3,Channel Islands,561821.666667,13.333333,4.9500,66.00000,14934.000000
4,Cyprus,553141.333333,2.333333,4.7500,10.95000,12373.333333


In [137]:
fig3 = px.scatter(dataPartyGroup, x='Quantity', y='UnitPrice', color='Country')
fig3.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [171]:
countries = ['EIRE','Germany','France','Netherlands']
dataHist = data[data['Country'].isin(countries)]
dataHist.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.8,14911,EIRE
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.8,14911,EIRE
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE


In [172]:
table = pd.pivot_table(dataHist, values='Quantity', index='InvoiceNo',
                    columns='Country', aggfunc='sum')
#dataHistGroup = dataHist.groupby(['Country','InvoiceNo']).sum()
#dataHistGroup
table.iplot(subplots=True, kind='histogram', xTitle='InvoiceNo', yTitle='Quantity')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [173]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [180]:
dataRevenue = data[data['Country'].isin(country_list)]
dataRevenue = data[data['Description'].isin(product_list)]
dataRevenue.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [181]:
fig5 = go.Figure(data=[
    go.Bar(name='CREAM HANGING HEART T-LIGHT HOLDER', x=dataRevenue.Country, y=dataRevenue.Revenue),
    go.Bar(name='JUMBO BAG RED RETROSPOT', x=dataRevenue.Country, y=dataRevenue.Revenue),
    go.Bar(name='REGENCY CAKESTAND 3 TIER', x=dataRevenue.Country, y=dataRevenue.Revenue)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [183]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [185]:
datauk = uk.groupby(['Day','Month','Year']).sum()
datauk = datauk.reset_index()
datauk.head()

,Day,Month,Year,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
0,1,2,2011,597138837,13536,3286.39,23376.20,17359973
1,1,3,2011,430780337,7614,2575.73,12668.97,12156686
2,1,4,2011,564531409,17149,3086.55,22052.60,15337564
3,1,5,2011,228882435,3172,1210.79,5879.33,6347810
4,1,6,2011,352010976,6091,1868.76,9207.03,10013779


In [188]:
fig6 = px.line(datauk, x='Day', y="Quantity", title='Quantity per Day')
fig6.show()

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [200]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)
products = products.reset_index()

In [201]:
fig7 = px.scatter(products, x="InvoiceNo", y="CustomerID")
fig7.show()

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [202]:
products.head()

,Description,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
0,4 PURPLE FLOCK DINNER CANDLES,35,134,2.318421,255.46,30
1,50'S CHRISTMAS GIFT BAG LARGE,100,1721,1.247900,2067.25,98
2,DOLLY GIRL BEAKER,100,661,1.250000,826.25,77
3,I LOVE LONDON MINI BACKPACK,55,181,4.150000,751.15,46
4,NINE DRAWER OFFICE TIDY,25,44,14.761538,628.40,24


In [203]:
fig8 = px.bar(products, x='Description', y='Revenue')
fig8.show()